# Testing access to Azure OpenAI through Frontdoor and API Management

### Create deployment using Terraform

This lab uses Terraform to declaratively define all the resources that will be deployed. Change the [variables.tf](variables.tf) directly to try different configurations.

In [ ]:
! terraform init
! terraform apply -auto-approve

### Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [1]:
apim_resource_gateway_url = ! terraform output -raw apim_resource_gateway_url
apim_resource_gateway_url = apim_resource_gateway_url.n
print("👉🏻 APIM Resource Gateway URL: ", apim_resource_gateway_url)

apim_subscription_key = ! terraform output -raw apim_subscription_key
apim_subscription_key = apim_subscription_key.n
print("👉🏻 APIM Subscription Key: ", apim_subscription_key)

frontdoor_hostname = ! terraform output -raw frontdoor_hostname
frontdoor_hostname = frontdoor_hostname.n
print("👉🏻 Front Door Hostname: ", frontdoor_hostname)

openai_api_version = "2024-10-21"
openai_model_name = "gpt-4o"
openai_deployment_name = "gpt-4o"

👉🏻 APIM Resource Gateway URL:  https://apim-genai-std-v2-18833-361.azure-api.net
👉🏻 APIM Subscription Key:  0f9bd909f6194ae491759bcc6cf90ab0
👉🏻 Front Door Hostname:  endpoint-apim-c9f6f2hdetd0aqfh.b01.azurefd.net


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [9]:
import json
import requests

url = "https://" + frontdoor_hostname + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
# url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}

response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)

# Check the response status code and apply formatting
if 200 <= response.status_code < 300:
    status_code_str = '\x1b[1;32m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and green
elif response.status_code >= 400:
    status_code_str = '\x1b[1;31m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and red
else:
    status_code_str = str(response.status_code)  # No formatting
    
# Print the response status with the appropriate formatting
print("Response status:", status_code_str)

if (response.status_code == 200):
    data = json.loads(response.text)
    print("Token usage:", data.get("usage"), "\n")
    print("💬 ", data.get("choices")[0].get("message").get("content"), "\n")
else:
    print(response.text)   

Response status: 200 - OK
Token usage: {'completion_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 30, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 78} 

💬  Oh, absolutely! Let me just check my invisible watch here... Yup, it’s exactly half past a freckle. Why don't you try using a clock or your phone, you know, those inventions designed specifically to tell time? 



<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [8]:
from openai import AzureOpenAI

messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

client = AzureOpenAI(
    azure_endpoint="https://" + frontdoor_hostname,
    api_key=apim_subscription_key,
    api_version=openai_api_version
)

response = client.chat.completions.create(model=openai_model_name, messages=messages)

print("💬 ", response.choices[0].message.content)

💬  Sure, because I have the magical ability to know your local time without any internet connection. Actually, you'll have to look at a clock yourself.
